In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/disc-data/disc_6.csv
/kaggle/input/all-discord-data/combined_df.csv


In [2]:
! pip install -U accelerate
! pip install datasets transformers
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [3]:
from huggingface_hub import notebook_login, login

notebook_login()

In [4]:
import transformers

print(transformers.__version__)

4.38.1


In [5]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
!pip install evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 806.9 kB/s eta 0:00:000:00:01


In [7]:
!ls -la

total 12
drwxr-xr-x 3 root root 4096 Mar 14 03:35 .
drwxr-xr-x 5 root root 4096 Mar 14 03:35 ..
drwxr-xr-x 2 root root 4096 Mar 14 03:35 .virtual_documents


In [20]:
df = pd.read_csv("/kaggle/input/disc_6.csv").dropna().drop("Author_ID", axis=1)
# df_2 = pd.read_csv("/content/drive/MyDrive/Data/disc_data/disc_7.csv")

train, test = train_test_split(df, test_size=0.2)
#test, val = train_test_split(test, test_size=0.5)

In [21]:
train.head()

,Message
704932,:o\nshow!
442973,Fuc
639576,MRS OBAMA GET DOWN
387338,I DON'T KNOW\nIM SORRY
690011,you cant hide the truth!


In [22]:
test.head()

,Message
595776,fren answer me
209890,how do u know
676675,IDK A TOBY
125952,This is all of us
305821,Since its igns job to rate\nI think they overrate


In [23]:
train_ds = Dataset.from_pandas(train)
test_ds = Dataset.from_pandas(test)
#validation_ds = Dataset.from_pandas(val)

datasets = DatasetDict({
    'train': train_ds,
    'test': test_ds,
    #'valid': validation_ds
})

In [24]:
datasets["train"][11]

{'Message': '5 admins geez', '__index_level_0__': 132385}

In [15]:
!git config --global user.email "emailsiddha@gmail.com"
!git config --global user.name "Siddharth Gupta"

In [18]:
from transformers import AutoTokenizer


model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [25]:
datasets

DatasetDict({
    train: Dataset({
        features: ['Message', '__index_level_0__'],
        num_rows: 586284
    })
    test: Dataset({
        features: ['Message', '__index_level_0__'],
        num_rows: 146572
    })
})

In [27]:
def tokenize_function(examples):
    result = tokenizer(examples["Message"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, remove_columns=["Message","__index_level_0__"]
)
tokenized_datasets

  0%|          | 0/587 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1141 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/147 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 586284
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 146572
    })
})

In [40]:
chunk_size = 32

In [41]:
tokenized_samples = tokenized_datasets["train"][:3]

In [42]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()}

In [43]:
total_length = len(concatenated_examples["input_ids"])
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 18'


In [44]:
train.describe()

,Message
count,586284
unique,431904
top,Welcome
freq,4751
